In [1]:
from collections import Counter
import os
import re
from typing import List, Tuple, Dict, Union
import unicodedata as ud

import fire
import pandas as pd
import numpy as np
Num = Union[int, float]
STOPPOS_JP = ["形容動詞語幹", "副詞可能", "代名詞", "ナイ形容詞語幹", "特殊", "数", "接尾", "非自立"]

In [2]:
with open(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/stopwords_jp.txt"), "r") as f:
    STOPWORDS_JP = [line.strip() for line in f]
STOPPOS_JP = ["形容動詞語幹", "副詞可能", "代名詞", "ナイ形容詞語幹", "特殊", "数", "接尾", "非自立"]

with open(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/AWD-J_EX.txt"), "r") as f:
    rows = [line.strip().split("\t") for line in f]
    AWD = {word: score for word, score, _, _ in rows}
DF_jiwc = pd.read_csv(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/2017-11-JIWC.csv"), index_col=1).drop(
    columns="Unnamed: 0"
)

In [3]:
import spacy
import ginza
NLP = spacy.load('ja_ginza')

In [4]:
def measure_pos_ginza(text: str) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            token_inflection = re.split('[,-]', ginza.inflection(token))
            analysis = token_tag + token_inflection
            analysis.append(token.lemma_)
            analysis += re.split('[,-]', ginza.reading_form(token))
            tuple_ = (token.orth_, analysis)
            tokens.append(tuple_)
    
    # VERB RELATED MEASURES
    verbs = [token for token in tokens if token[1][0] == "動詞"]
    
    # CONTENT WORDS RATIO
    nouns = []
    for token in tokens:
        match = re.search(r"名詞$", token[1][0])
        if match is not None:
            nouns.append(token)
    adjcs = [token for token in tokens if token[1][0] == "形容詞"]
    content_words = verbs + nouns + adjcs
    cwr_simple = np.divide(len(content_words), len(tokens))
    cwr_advance = np.divide(
        len(
            [
                token
                for token in content_words
                if (token[1][1] not in STOPPOS_JP) and (token[0] not in STOPWORDS_JP)
            ]
        ),
        len(tokens),
    )

    # Modifying words and verb ratio (MVR)
    advbs = [token for token in tokens if token[1][0] == "副詞"]
    padjs = [token for token in tokens if token[1][0] == "連体詞"]
    mvr = np.divide(len(adjcs + advbs + padjs), len(verbs))

    # NER
    ners = [token for token in tokens if token[1][1] == "固有名詞"]
    nerr = np.divide(len(ners), len(tokens))

    # TTR
    ttrs = 0 #calc_ttrs(tokens)

    return verbs, nouns, adjcs, content_words, cwr_simple, cwr_advance, advbs, mvr, padjs, nerr, ttrs

In [7]:
text = 'フランスのU-24代表は、本来であれば候補である若手選手が、EURO2020やクラブ事情の関係で招集できず、メンバーが二転三転したことは周知の通りだ。とはいえ、オリンピックの舞台で、フランスというサッカー強豪国の若い世代が大敗したことは、少なからず衝撃をもって受け止められている。'

In [8]:
measure_pos_ginza(text)

([('あれ', ['動詞', '非自立可能', '五段', 'ラ行', '仮定形', '一般', 'ある', 'アレ']),
  ('ある', ['動詞', '非自立可能', '五段', 'ラ行', '連体形', '一般', 'ある', 'アル']),
  ('でき', ['動詞', '非自立可能', '上一段', 'カ行', '未然形', '一般', 'できる', 'デキ']),
  ('し', ['動詞', '非自立可能', 'サ行変格', '連用形', '一般', 'する', 'シ']),
  ('いえ', ['動詞', '一般', '文語四段', 'ハ行', '已然形', '一般', 'いう', 'イエ']),
  ('いう', ['動詞', '一般', '五段', 'ワア行', '連体形', '一般', 'いう', 'イウ']),
  ('し', ['動詞', '非自立可能', 'サ行変格', '連用形', '一般', 'する', 'シ']),
  ('もっ', ['動詞', '一般', '五段', 'タ行', '連用形', '促音便', 'もつ', 'モッ']),
  ('受け止め', ['動詞', '一般', '下一段', 'マ行', '未然形', '一般', '受け止める', 'ウケトメ']),
  ('いる', ['動詞', '非自立可能', '上一段', 'ア行', '終止形', '一般', 'いる', 'イル'])],
 [('フランス', ['名詞', '固有名詞', '地名', '国', '', 'フランス', 'フランス']),
  ('U', ['名詞', '普通名詞', '一般', '', 'u', 'U']),
  ('24', ['名詞', '数詞', '', '24', 'ニヨン']),
  ('代表', ['名詞', '普通名詞', 'サ変可能', '', '代表', 'ダイヒョウ']),
  ('本来', ['名詞', '普通名詞', '副詞可能', '', '本来', 'ホンライ']),
  ('候補', ['名詞', '普通名詞', '一般', '', '候補', 'コウホ']),
  ('若手', ['名詞', '普通名詞', '一般', '', '若手', 'ワカテ']),
  ('選手', ['名詞', '普通名

In [9]:
def measure_pos_ginza(text: str) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            token_inflection = re.split('[,-]', ginza.inflection(token))
            analysis = token_tag + token_inflection
            analysis.append(token.lemma_)
            analysis += re.split('[,-]', ginza.reading_form(token))
            tuple_ = (token.orth_, analysis)
            tokens.append(tuple_)
    
    # VERB RELATED MEASURES
    verbs = [token for token in tokens if token[1][0] == "動詞"]
    
    # CONTENT WORDS RATIO
    nouns = []
    for token in tokens:
        match = re.search(r"名詞$", token[1][0])
        if match is not None:
            nouns.append(token)
    adjcs = [token for token in tokens if token[1][0] == "形容詞"]
    content_words = verbs + nouns + adjcs
    cwr_simple = np.divide(len(content_words), len(tokens))
    cwr_advance = np.divide(
        len(
            [
                token
                for token in content_words
                if (token[1][1] not in STOPPOS_JP) and (token[0] not in STOPWORDS_JP)
            ]
        ),
        len(tokens),
    )

    # Modifying words and verb ratio (MVR)
    advbs = [token for token in tokens if token[1][0] == "副詞"]
    padjs = [token for token in tokens if token[1][0] == "連体詞"]
    mvr = np.divide(len(adjcs + advbs + padjs), len(verbs))

    # NER
    ners = [token for token in tokens if token[1][1] == "固有名詞"]
    nerr = np.divide(len(ners), len(tokens))

    # TTR
    ttrs = 0 #calc_ttrs(tokens)

    return tokens
measure_pos_ginza(text)

[('フランス', ['名詞', '固有名詞', '地名', '国', '', 'フランス', 'フランス']),
 ('の', ['助詞', '格助詞', '', 'の', 'ノ']),
 ('U', ['名詞', '普通名詞', '一般', '', 'u', 'U']),
 ('-', ['補助記号', '一般', '', '－', 'キゴウ']),
 ('24', ['名詞', '数詞', '', '24', 'ニヨン']),
 ('代表', ['名詞', '普通名詞', 'サ変可能', '', '代表', 'ダイヒョウ']),
 ('は', ['助詞', '係助詞', '', 'は', 'ハ']),
 ('、', ['補助記号', '読点', '', '、', '、']),
 ('本来', ['名詞', '普通名詞', '副詞可能', '', '本来', 'ホンライ']),
 ('で', ['助動詞', '助動詞', 'ダ', '連用形', '一般', 'だ', 'デ']),
 ('あれ', ['動詞', '非自立可能', '五段', 'ラ行', '仮定形', '一般', 'ある', 'アレ']),
 ('ば', ['助詞', '接続助詞', '', 'ば', 'バ']),
 ('候補', ['名詞', '普通名詞', '一般', '', '候補', 'コウホ']),
 ('で', ['助動詞', '助動詞', 'ダ', '連用形', '一般', 'だ', 'デ']),
 ('ある', ['動詞', '非自立可能', '五段', 'ラ行', '連体形', '一般', 'ある', 'アル']),
 ('若手', ['名詞', '普通名詞', '一般', '', '若手', 'ワカテ']),
 ('選手', ['名詞', '普通名詞', '一般', '', '選手', 'センシュ']),
 ('が', ['助詞', '格助詞', '', 'が', 'ガ']),
 ('、', ['補助記号', '読点', '', '、', '、']),
 ('EURO', ['名詞', '普通名詞', '助数詞可能', '', 'euro', 'ユーロ']),
 ('2020', ['名詞', '数詞', '', '2020', 'ニレイニレイ']),
 ('や', ['助詞',

In [17]:
def verbs(text: str) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            tuple_ = (token.orth_, token_tag)
            tokens.append(tuple_)
    
    # VERB RELATED MEASURES
    verbs = [token for token in tokens if token[1][0] == "動詞"]
    
    return tokens
verbs(text)

[('フランス', ['名詞', '固有名詞', '地名', '国']),
 ('の', ['助詞', '格助詞']),
 ('U', ['名詞', '普通名詞', '一般']),
 ('-', ['補助記号', '一般']),
 ('24', ['名詞', '数詞']),
 ('代表', ['名詞', '普通名詞', 'サ変可能']),
 ('は', ['助詞', '係助詞']),
 ('、', ['補助記号', '読点']),
 ('本来', ['名詞', '普通名詞', '副詞可能']),
 ('で', ['助動詞']),
 ('あれ', ['動詞', '非自立可能']),
 ('ば', ['助詞', '接続助詞']),
 ('候補', ['名詞', '普通名詞', '一般']),
 ('で', ['助動詞']),
 ('ある', ['動詞', '非自立可能']),
 ('若手', ['名詞', '普通名詞', '一般']),
 ('選手', ['名詞', '普通名詞', '一般']),
 ('が', ['助詞', '格助詞']),
 ('、', ['補助記号', '読点']),
 ('EURO', ['名詞', '普通名詞', '助数詞可能']),
 ('2020', ['名詞', '数詞']),
 ('や', ['助詞', '副助詞']),
 ('クラブ', ['名詞', '普通名詞', '一般']),
 ('事情', ['名詞', '普通名詞', '一般']),
 ('の', ['助詞', '格助詞']),
 ('関係', ['名詞', '普通名詞', 'サ変可能']),
 ('で', ['助詞', '格助詞']),
 ('招集', ['名詞', '普通名詞', 'サ変可能']),
 ('でき', ['動詞', '非自立可能']),
 ('ず', ['助動詞']),
 ('、', ['補助記号', '読点']),
 ('メンバー', ['名詞', '普通名詞', '一般']),
 ('が', ['助詞', '格助詞']),
 ('二転', ['名詞', '普通名詞', 'サ変可能']),
 ('三転', ['名詞', '普通名詞', 'サ変可能']),
 ('し', ['動詞', '非自立可能']),
 ('た', ['助動詞']),
 ('こと', [